In [6]:
#Load dependencies
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
#Define constants
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [7]:
#Define our neural network mapping observations to actions
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()#Call init function of the underlying class to properly initialize Net as a nn.Module
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [8]:
#Episode saves total reward of episode and a list of EpisodeSteps
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
#EpisodeStep saves the given observation and the action choosen by our net at every step
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [9]:
def iterate_batches(env, net, batch_size):
    batch = [] #Initialize empty batch_list
    episode_reward = 0.0 #Initialize reward of episode
    episode_steps = [] #Initalize empty step list
    obs, _ = env.reset() #Reset environment
    sm = nn.Softmax(dim=1)#Define softmax function
    while True:
        obs_v = torch.FloatTensor([obs])#Convert observations to tensor with batch_dimension (hence list)
        act_probs_v = sm(net(obs_v)) #Apply softmax to NN-output to get probas
        act_probs = act_probs_v.data.numpy()[0] #Extract probas by converting to numpy and removing batch_dimension
        action = np.random.choice(len(act_probs), p=act_probs) #Choose random action
        next_obs, reward, is_terminated, is_truncated, _ = env.step(action) #Make environment step
        episode_reward += reward #Accumulate reward for episode
        episode_steps.append(EpisodeStep(observation=obs, action=action)) #Log EpisodeStep
        if is_terminated or is_truncated:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))#Append statistics from entire episode
            episode_reward = 0.0 #Reset reward
            episode_steps = [] #Reset steps
            next_obs, _ = env.reset() #Reset env
            if len(batch) == batch_size: #If batch_size is reached
                yield batch #Return generator for training
                batch = [] #Reset batch
        obs = next_obs #Update observation

In [10]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))#NOT SURE!!!
    reward_bound = np.percentile(rewards, percentile)#Calculate bound of reward
    reward_mean = float(np.mean(rewards)) #Calculate mean reward

    train_obs = []
    train_act = []
    for example in batch:
        if example.reward < reward_bound: #Only if reward exceeds the bound set by percentile
            continue
        train_obs.extend(map(lambda step: step.observation, example.steps))
        train_act.extend(map(lambda step: step.action, example.steps))

    train_obs_v = torch.FloatTensor(train_obs) #Convert observations to tensor
    train_act_v = torch.LongTensor(train_act) #Convert actions to tensor
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [11]:
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n
print(f"Number of actions: {n_actions}")

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

/opt/conda/lib/python3.10/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


Number of actions: 2


/tmp/ipykernel_24/1921783845.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  obs_v = torch.FloatTensor([obs])#Convert observations to tensor with batch_dimension (hence list)
/opt/conda/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


0: loss=0.708, reward_mean=20.7, reward_bound=22.5
1: loss=0.689, reward_mean=27.7, reward_bound=35.5
2: loss=0.672, reward_mean=24.5, reward_bound=31.5
3: loss=0.663, reward_mean=26.7, reward_bound=25.0
4: loss=0.647, reward_mean=33.8, reward_bound=39.0
5: loss=0.647, reward_mean=28.0, reward_bound=31.0
6: loss=0.632, reward_mean=39.4, reward_bound=40.5
7: loss=0.614, reward_mean=43.4, reward_bound=52.5
8: loss=0.618, reward_mean=49.1, reward_bound=54.0
9: loss=0.600, reward_mean=56.6, reward_bound=64.0
10: loss=0.612, reward_mean=58.8, reward_bound=61.0
11: loss=0.611, reward_mean=40.9, reward_bound=47.0
12: loss=0.588, reward_mean=58.0, reward_bound=71.0
13: loss=0.579, reward_mean=52.2, reward_bound=63.5
14: loss=0.564, reward_mean=68.6, reward_bound=82.0
15: loss=0.588, reward_mean=61.9, reward_bound=70.0
16: loss=0.557, reward_mean=70.6, reward_bound=90.0
17: loss=0.575, reward_mean=59.7, reward_bound=77.0
18: loss=0.567, reward_mean=63.8, reward_bound=65.5
19: loss=0.577, reward

In [12]:
%load_ext tensorboard
%tensorboard --logdir=runs --host=0.0.0.0 --port=6006